In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import os
import glob 
from shapely.wkt import loads
import warnings
warnings.filterwarnings('ignore')

### 读取充电站数据

In [3]:
buffer_radius = 500  #搜索充电站周边500m以内得建筑物信息
charge_station = gpd.read_file('./data/上海充电桩_中促盟_处理后.geojson')
charge_station.rename(columns={'经度':'lon','纬度':'lat'},inplace=True)
charge_station['station_sum'] = charge_station['交流桩数量'] + charge_station['直流桩数量']
## 删除没用的充电站类型数据
charge_station = charge_station[charge_station['充电站类型']!='04环卫物流车专用']
## lon, lat统一使用wgs 84 坐标系
charge_station.crs = {'init':'epsg:4326'}
## 使用适配于上海的投影坐标系
charge_station_proj = charge_station.to_crs(epsg=4528)
charge_station['geometry'] = charge_station_proj.buffer(buffer_radius)
charge_station.to_crs(epsg=4326,inplace=True)

### 读取上海建筑物面积

In [3]:
shanghai_area = pd.read_csv('./data/shanghai.csv')
## 转换成polygon数据
shanghai_area['geometry'] = shanghai_area['geometry'].apply(lambda x: loads(x) if isinstance(x, object) else None)
## dataframe转换成geodataframe
shanghai_area = gpd.GeoDataFrame(shanghai_area,geometry='geometry')
shanghai_area.drop(columns=['index_right'],inplace=True)
## 计算建筑物面积
def building_area(gdf):
    gdf.crs = {'init':'epsg:4326'}
    ## 使用适配于上海的投影坐标系
    tmp = gdf.to_crs(epsg=4528)
    gdf['area'] = tmp.area
    ## 计算总面积
    gdf['area'] = gdf['area'] * gdf['height']
    return gdf
shanghai_area = building_area(shanghai_area)

### 合并建筑物信息和充电站数据信息

In [4]:
station_sjoined_area = gpd.sjoin(charge_station,shanghai_area)
station_sjoined_area.drop(columns=['index_right','building_id'],inplace=True)
station_sjoined_area = station_sjoined_area.groupby(['myid'])['area'].sum().reset_index()
station_sjoined_area

,myid,area
0,0,4.578910e+05
1,1,6.692213e+05
2,2,1.414586e+06
3,3,4.275121e+05
4,4,4.275121e+05
...,...,...
19552,19611,4.983789e+06
19553,19612,4.492931e+06
19554,19613,1.616420e+06
19555,19615,5.387258e+05


### 把建筑物面积合并到充电站

In [5]:
charge_station_merge = pd.merge(charge_station,station_sjoined_area,on=['myid'],how='left')
charge_station_merge['area'] = charge_station_merge['area'].fillna(0)
charge_station_merge

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,geometry,station_sum,area
0,0,上海金山石化十五村充电项目,上海市市辖区金山区石化街道金山城市沙滩石化十五村,2023-09-27,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,0,"POLYGON ((121.35313 30.71258, 121.35310 30.712...",6.0,4.578910e+05
1,1,上海金山石化十一村小区充电站,上海市市辖区金山区石化街道金山城市沙滩石化十一村,2023-09-27,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,0,"POLYGON ((121.35336 30.71514, 121.35332 30.714...",6.0,6.692213e+05
2,2,石化隆安市场停车场,石化街道沪杭公路7958号,2021-09-16,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,0,"POLYGON ((121.34978 30.71665, 121.34975 30.716...",12.0,1.414586e+06
3,3,上海欣鑫化工充电站,上海市市辖区金山区海金路366号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,2.0,1.0,14.0,60,"POLYGON ((121.30128 30.71718, 121.30125 30.716...",3.0,4.275121e+05
4,4,上海顶煦化学充电站,上海市市辖区金山区金山区海金路369号,2017-01-18,01城市内道路,121.296063,30.717229,01充电场站,01公共,5.0,1.0,35.0,15,"POLYGON ((121.30128 30.71718, 121.30125 30.716...",6.0,4.275121e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19585,19611,东方希杰地下车库内部员工新能源车充电站,国定路400号东方希杰地下车库,2023-08-11,None,121.508835,31.299307,01充电场站,01公共,2.0,0.0,14.0,0,"POLYGON ((121.51409 31.29925, 121.51405 31.298...",2.0,4.983789e+06
19586,19612,加枫路充电站,加枫路26号衡山商务停车场,2022-01-13,None,121.605119,31.313403,03公共机构内部停车场,01公共,0.0,4.0,0.0,240,"POLYGON ((121.61037 31.31334, 121.61034 31.312...",4.0,4.492931e+06
19587,19613,华润新江湾九里充电站,华润新江湾九里,2023-09-23,None,121.520188,31.338381,07交通枢纽公共停车场,01公共,108.0,0.0,756.0,0,"POLYGON ((121.52544 31.33832, 121.52541 31.337...",108.0,1.616420e+06
19588,19615,宝钱公路慢充站,宝钱公路,2023-11-27,None,121.119527,31.362807,05写字楼内部停车场,01公共,2.0,0.0,14.0,0,"POLYGON ((121.12478 31.36276, 121.12475 31.362...",2.0,5.387258e+05


### 读取上海的poi建筑信息

In [6]:
def read_poi(folder_path):
    csv_file_pattern = os.path.join(folder_path, '*.csv')  # 构建文件匹配模式
    csv_files = glob.glob(csv_file_pattern)  # 获取所有.csv文件的路径列表
    data = pd.DataFrame()
    for file_path in csv_files:
        df = pd.read_csv(file_path,on_bad_lines='skip')
        ## 处理tsv文件
        if len(df.columns) == 1:
            df = pd.read_csv(file_path,sep='\t')
        data = pd.concat([data,df])
    return data

In [7]:
poi1 = read_poi('./POI')
poi2 = read_poi('./POI2')
poi3 = read_poi('./POI长三角')

In [8]:
poi1 = poi1[poi1['province']=='上海市']
poi2 = poi2[poi2['province']=='上海市']
poi3 = poi3[poi3['province']=='上海市']

In [9]:
## 合并上海市所有的POI信息
poi_shanghai = pd.concat([poi1,poi2,poi3])
poi_shanghai.drop_duplicates(subset=['uid'],inplace=True)
poi_shanghai = gpd.GeoDataFrame(poi_shanghai,
                                geometry=gpd.points_from_xy(poi_shanghai['lon'],poi_shanghai['lat']))

In [10]:
tech_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('科技园')]
tech_park_poi['technology_park'] = 1
tech_park_poi = tech_park_poi[['technology_park','geometry']]

In [11]:
car_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('停车场')]
car_park_poi['car_park'] = 1
car_park_poi = car_park_poi[['car_park','geometry']]

In [12]:
supermarket_poi = poi_shanghai[(poi_shanghai['name'].str.contains('超市'))&(poi_shanghai['tag'].str.contains('超市'))]
supermarket_poi['supermarket'] = 1
supermarket_poi = supermarket_poi[['supermarket','geometry']]

In [13]:
tag_not_nan_poi = poi_shanghai.dropna(subset=['tag'])
residential_poi = tag_not_nan_poi[tag_not_nan_poi['tag'].str.contains('住宅区')]
residential_poi['residential'] = 1
residential_poi = residential_poi[['residential','geometry']]

In [14]:
commercial_poi = poi_shanghai[poi_shanghai['type']=='shopping']
commercial_poi['commercial'] = 1
commercial_poi = commercial_poi[['commercial','geometry']]

In [15]:
def merge_info_to_charge_station(charge_station,merge_poi,column):
    tmp_sjoined = gpd.sjoin(charge_station,merge_poi,how='left',op='intersects')
    tmp_sjoined[column] = tmp_sjoined[column].fillna(0)
    myid_agg = tmp_sjoined.groupby(['myid'])[column].sum().reset_index()
    return myid_agg

In [16]:
technology_agg = merge_info_to_charge_station(charge_station_merge,tech_park_poi,'technology_park')
car_agg = merge_info_to_charge_station(charge_station_merge,car_park_poi,'car_park')
supermarket_agg = merge_info_to_charge_station(charge_station_merge,supermarket_poi,'supermarket')
residential_agg = merge_info_to_charge_station(charge_station_merge,residential_poi,'residential')
commercial_agg = merge_info_to_charge_station(charge_station_merge,commercial_poi,'commercial')

In [17]:
geographic_info = pd.merge(technology_agg,car_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,supermarket_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,residential_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,commercial_agg,on=['myid'],how='inner')
## 合并地理信息
charge_station_feature = pd.merge(charge_station_merge,geographic_info,on=['myid'],how='inner')
charge_station_feature.drop(columns=['充电站位置', '充电站投入使用时间'],inplace=True)

In [18]:
charge_station_feature.to_csv('./output/中促盟充电站特征.csv',index=None)

### 栅格和充电站对应

### 读取所有的栅格级订单数据并且生成栅格范围内geometry信息

In [18]:
charge_grid_1 = pd.read_csv('data/订单量0712.csv')
charge_grid_2 = pd.read_csv('data/订单量0715.csv')
charge_grid = pd.concat([charge_grid_1,charge_grid_2])
charge_grid.drop_duplicates(subset=['经纬度编号'],inplace=True)
# 生成栅格范围
def generate_grid(grid_heatmap):
    grid_heatmap['lon'] = grid_heatmap['经纬度编号'].str.split('-').str[0].astype(float)
    grid_heatmap['lat'] = grid_heatmap['经纬度编号'].str.split('-').str[1].astype(float)
    params = {'slon': 120,
    'slat': 30,
    'deltalon': 0.01,
    'deltalat': 0.01,
    'theta': 0,
    'method': 'rect',
    'gridsize': 500}
    grid_heatmap['LONCOL'],grid_heatmap['LATCOL'] = tbd.GPS_to_grid(grid_heatmap['lon'], grid_heatmap['lat'],params)
    grid_heatmap['geometry'] = tbd.grid_to_polygon([grid_heatmap['LONCOL'], grid_heatmap['LATCOL']],params)
    grid_heatmap = gpd.GeoDataFrame(grid_heatmap, geometry='geometry')
    return grid_heatmap
Charge_Grid = generate_grid(charge_grid)

In [19]:
grid_shanghai = Charge_Grid.drop_duplicates(subset=['lon','lat'])
grid_shanghai['index'] = [i for i in range(len(grid_shanghai))]
gdf_grid_shanghai = gpd.GeoDataFrame(
    grid_shanghai, 
    geometry='geometry'  # 指定几何列的名称
)
gdf_grid_shanghai.crs = {'init':'epsg:4326'}
gdf_grid_shanghai[['lon','lat','index']].to_csv('./output/grid_index.csv',index=None)

In [20]:
## 找到每个充电站uid对应得栅格
charge_station_1 = gpd.read_file('./data/上海充电桩_中促盟_处理后.geojson')
grid_sjoined_charge_station = gpd.sjoin(charge_station_1,gdf_grid_shanghai)
grid_sjoined_charge_station = grid_sjoined_charge_station[['myid','index']]

In [ ]:
charge_station_feature_grid = pd.merge(charge_station_feature,grid_sjoined_charge_station,on=['myid'],how='inner')
charge_station_feature_grid.drop(columns=['geometry'])
charge_station_feature_grid.to_csv('./output/charge_station_feature.csv',index=None)